In [109]:
import string
import random
import nltk
from nltk.corpus import reuters
from nltk import FreqDist
from nltk.corpus import stopwords

In [110]:
from nltk.corpus import stopwords
import random
from sklearn.model_selection import train_test_split
import math

In [111]:
import pandas as pd
df=pd.read_csv("preprocessed_data.csv")

In [112]:
df

,Unnamed: 0.1,Unnamed: 0,Post_ID,Comment_ID,Comment,Majority_Sentiment,lemmatized_comments
0,0,1,108l3ho,j3vlnat,Some of them yes but this one i got it from my...,Neutral,some of them yes but this one i got it from my...
1,1,2,10j6oqj,j5j34zb,I keep mine in my phone cover for good luck,Positive,i keep mine in my phone cover for good luck
2,2,3,10j6oqj,j5jbs3f,Hang on to it These are rare to come by as the...,Neutral,hang on to it these are rare to come by a the...
3,3,4,10j6oqj,j5jfczd,Yeah Cool I got this from my classmate in 2015...,Positive,yeah cool i got this from my classmate in 2...
4,4,5,10o9tvi,j6h1ko8,Check out the Kaja Throm Beautiful marketplace...,Positive,check out the kaja throm beautiful marketplac...
...,...,...,...,...,...,...,...
130611,130611,166838,zp46e8,0,Our population growth rate is 19 Our Birth rat...,Neutral,our population growth rate is 19 our birth ra...
130612,130612,166839,zp46e8,0,I obviously wont But not only you dont do bes...,Negative,i obviously wo nt but not only you do nt do ...
130613,130613,166840,zp46e8,0,bengal has always been some of the most fertil...,Positive,bengal ha always been some of the most fertile...
130614,130614,166841,zp46e8,0,racist agenda\n\nAhhh yes its always racism\n\...,Negative,racist agenda ahhh yes it s always racism t...


In [113]:
proc_sents=[sentence for sentence in df.loc[:,"lemmatized_comments"]]

In [114]:
for sen in proc_sents:
    if 'http' in sen:
        proc_sents.remove(sen)

In [115]:
proc_sents[0:2]

['some of them yes but this one i got it from my brother who studied in malaysia and he made a friend from bhutan there and he found it for me',
 'i keep mine in my phone cover for good luck']

In [116]:
for i in range(len(proc_sents)):
    sentence=proc_sents[i]
    proc_sents[i]=sentence.split()

In [117]:
proc_sents[0:2]

[['some',
  'of',
  'them',
  'yes',
  'but',
  'this',
  'one',
  'i',
  'got',
  'it',
  'from',
  'my',
  'brother',
  'who',
  'studied',
  'in',
  'malaysia',
  'and',
  'he',
  'made',
  'a',
  'friend',
  'from',
  'bhutan',
  'there',
  'and',
  'he',
  'found',
  'it',
  'for',
  'me'],
 ['i', 'keep', 'mine', 'in', 'my', 'phone', 'cover', 'for', 'good', 'luck']]

In [118]:
import numpy as np
np.random.seed(0)
train_set, validation_set = train_test_split(proc_sents, test_size=0.2)

In [119]:
word_to_int = {}
count = 0
for sentence in train_set:
    for token in sentence:
        if(token not in word_to_int):
            word_to_int[token] = count
            count += 1

In [120]:
unigrams = {}
bigrams = {}
trigrams = {}
quadgrams = {}

In [121]:
for sentence in train_set:
    for token in sentence:
        if(token not in unigrams): unigrams[token] = 0
        unigrams[token] += 1

In [122]:
# word_to_int
print({k: word_to_int[k] for k in list(word_to_int)[:3]})

{'since': 0, 'you': 1, 'are': 2}


In [123]:
for sentence in train_set:
    for i in range(3 , len(sentence)):
        quadgram = (sentence[i-3] , sentence[i-2] , sentence[i-1], sentence[i])
        if(quadgram not in quadgrams): quadgrams[quadgram] = 0
        quadgrams[quadgram] += 1

In [124]:
for sentence in train_set:
    for i in range(2 , len(sentence)):
        trigram = (sentence[i-2] , sentence[i-1], sentence[i])
        if(trigram not in trigrams): trigrams[trigram] = 0
        trigrams[trigram] += 1

In [125]:
for sentence in train_set:
    for i in range(1 , len(sentence)):
        bigram = (sentence[i-1], sentence[i])
        if(bigram not in bigrams): bigrams[bigram] = 0
        bigrams[bigram] += 1

In [126]:
# unigrams
print({k: unigrams[k] for k in list(unigrams)[:3]})

{'since': 1979, 'you': 52791, 'are': 32857}


In [127]:
# bigrams
print({k: bigrams[k] for k in list(bigrams)[:3]})

{('since', 'you'): 134, ('you', 'are'): 3861, ('are', 'bringing'): 17}


In [128]:
# trigrams
print({k: trigrams[k] for k in list(trigrams)[:3]})

{('since', 'you', 'are'): 18, ('you', 'are', 'bringing'): 7, ('are', 'bringing', 'up'): 4}


In [129]:
# quadgrams
print({k: quadgrams[k] for k in list(quadgrams)[:3]})

{('since', 'you', 'are', 'bringing'): 1, ('you', 'are', 'bringing', 'up'): 3, ('are', 'bringing', 'up', 'different'): 1}


In [130]:
## unigrams
total_sum = sum(unigrams.values())
total_perp = 0
count_sent = 0

for sentence in validation_set:

    count_sent += 1
    if(len(sentence) < 1): continue
    else:
        perp = 1
        n = len(sentence)
        log_perp = 0
        for i in range(1 , n):
            n_gram = ( sentence[i])

            if(n_gram[0] not in unigrams) :
                log_perp += math.log(total_sum)

            else:
                log_perp += math.log(total_sum/unigrams[n_gram[0]])
        log_perp = (1/n)*log_perp
        total_perp += math.exp(log_perp)
print("average perplexity of bigram is: ",total_perp / count_sent)

average perplexity of bigram is:  2431.529369177447


In [131]:
total_sum = sum(unigrams.values())
total_perp = 0
count_sent = 0

for sentence in validation_set:
    count_sent += 1
    if len(sentence) < 1:
        continue
    else:
        perp = 1
        n = len(sentence)
        log_perp = 0
        for i in range(n):
            n_gram = sentence[i]

            if n_gram[0] in unigrams:
                smooth_prob = ((total_sum + len(unigrams))/(unigrams[n_gram[0]] + 1))
                log_perp += math.log(smooth_prob)
            else:
                log_perp += math.log(1 / (total_sum + len(unigrams)))

        log_perp = (1 / n) * log_perp
        total_perp += math.exp(log_perp)

print("Average perplexity of Laplace smoothed unigram is:", total_perp / count_sent)


Average perplexity of Laplace smoothed unigram is: 2652.872424332836


In [132]:
## Bigrams
total_perp = 0
count_sent = 0

for sentence in validation_set:

    count_sent += 1
    if(len(sentence) < 2): continue
    else:
        perp = 1
        n = len(sentence)
        log_perp = 0
        for i in range(1 , n):
            n_gram = (sentence[i-1], sentence[i])

            if(n_gram[0] not in unigrams) or (n_gram not in bigrams):
                log_perp += math.log(len(unigrams))

            else:
                log_perp += math.log(unigrams[n_gram[0]]/(bigrams[n_gram]))
        log_perp = (1/n)*log_perp
        total_perp += math.exp(log_perp)
print("average perplexity of bigram is: ",total_perp / count_sent)

average perplexity of bigram is:  592.2785718972964


In [133]:
## Bigram Laplace smoothing

total_perp = 0
count_sent = 0

for sentence in validation_set:
#     print(sentence)

    count_sent += 1
    if(len(sentence) < 2): continue
    else:
        perp = 1
        n = len(sentence)
        log_perp = 0
        for i in range(1 , n):
            n_gram = (sentence[i-1], sentence[i])

            smooth_prob = (unigrams.get(n_gram[0],0) + len(unigrams))/(bigrams.get(n_gram,0)+1)
            log_perp += math.log(smooth_prob)

        log_perp = (1/n)*log_perp

        total_perp += math.exp(log_perp)
print("average perplexity of Laplace smoothened bigram is: ",total_perp / count_sent)

average perplexity of Laplace smoothened bigram is:  2950.6115686115413


In [134]:
## Trigram
total_perp = 0
count_sent = 0

for sentence in validation_set:
    count_sent += 1
    if(len(sentence) < 3): continue
    else:
        n = len(sentence)
        log_perp = 0
        for i in range(2 , n):
            n_gram = (sentence[i-2], sentence[i-1], sentence[i])

            if(n_gram[:-1] not in bigrams) or (n_gram not in trigrams):
                log_perp += math.log(len(unigrams))

            else:
                log_perp += math.log(bigrams[n_gram[:-1]]/(trigrams[n_gram]))
        log_perp = (1/n)*log_perp
        total_perp += math.exp(log_perp)
print("average perplexity of trigram is: ",total_perp / count_sent)

average perplexity of trigram is:  1973.644316192001


In [135]:
## Trigram LAPLAACE SMOOTHING

total_perp = 0
count_sent = 0

for sentence in validation_set:

    count_sent += 1
    if(len(sentence) < 3): continue
    else:
        n = len(sentence)
        log_perp = 0
        for i in range(2 , n):
            n_gram = (sentence[i-2], sentence[i-1], sentence[i])

            smooth_prob = (bigrams.get(n_gram[:-1],0) + len(unigrams))/(trigrams.get(n_gram,0)+1)
            log_perp += math.log(smooth_prob)

        log_perp = (1/n)*log_perp

        total_perp += math.exp(log_perp)
print("average perplexity of Laplace smoothened trigram is: ",total_perp / count_sent)

average perplexity of Laplace smoothened trigram is:  10617.441382254596


In [136]:
len(unigrams)

72406

In [137]:
## Quadgram
total_perp = 0
count_sent = 0

for sentence in validation_set:
#     print(sentence)
    count_sent += 1
    if(len(sentence) < 4): continue
    else:
        n = len(sentence)
        log_perp = 0
        for i in range(3 , n):
            n_gram = (sentence[i-3], sentence[i-2], sentence[i-1], sentence[i])

            if(n_gram[:-1] not in trigrams) or (n_gram not in quadgrams):
                log_perp += math.log(len(unigrams)) # 7 to 6
#                 print(len(quadgrams))

            else:
                log_perp += math.log(trigrams[n_gram[:-1]]/(quadgrams[n_gram]))
#                 print(trigrams[n_gram[:-1]]/(quadgrams[n_gram]), trigrams[n_gram[:-1]] , (quadgrams[n_gram]) )
        log_perp = (1/n)*log_perp
        total_perp += math.exp(log_perp)
print("average perplexity of quadgram is: ",total_perp / count_sent)

average perplexity of quadgram is:  6130.475783783798


In [138]:
## Quadgram LAPLAACE SMOOTHING

total_perp = 0
count_sent = 0

for sentence in validation_set:
#     print(sentence)

    count_sent += 1
    if(len(sentence) < 4): continue
    else:
        n = len(sentence)
        log_perp = 0
        for i in range(3 , n):
            n_gram = (sentence[i-3], sentence[i-2], sentence[i-1], sentence[i])

            smooth_prob = (trigrams.get(n_gram[:-1],0) + len(unigrams))/(quadgrams.get(n_gram,0)+1)
            log_perp += math.log(smooth_prob)

        log_perp = (1/n)*log_perp

        total_perp += math.exp(log_perp)
print("average perplexity of Laplace smoothened quadgram is: ",total_perp / count_sent)

average perplexity of Laplace smoothened quadgram is:  13924.504467870463


In [139]:
bigram_freq ={}

for frequency in bigrams.values():
    if frequency not in bigram_freq:
        bigram_freq[frequency] = 1
    else:
        bigram_freq[frequency] += 1


bigram_freq = dict(sorted(bigram_freq.items()))

bigram_freq

{1: 548511,
 2: 131693,
 3: 44380,
 4: 25590,
 5: 15691,
 6: 10828,
 7: 7964,
 8: 6031,
 9: 4792,
 10: 3875,
 11: 3155,
 12: 2561,
 13: 2301,
 14: 1995,
 15: 1782,
 16: 1514,
 17: 1403,
 18: 1210,
 19: 1103,
 20: 986,
 21: 900,
 22: 805,
 23: 712,
 24: 689,
 25: 593,
 26: 567,
 27: 514,
 28: 510,
 29: 440,
 30: 433,
 31: 415,
 32: 371,
 33: 349,
 34: 321,
 35: 259,
 36: 312,
 37: 278,
 38: 235,
 39: 244,
 40: 260,
 41: 217,
 42: 214,
 43: 214,
 44: 217,
 45: 183,
 46: 160,
 47: 174,
 48: 180,
 49: 156,
 50: 144,
 51: 160,
 52: 138,
 53: 121,
 54: 137,
 55: 136,
 56: 109,
 57: 110,
 58: 104,
 59: 97,
 60: 112,
 61: 112,
 62: 98,
 63: 99,
 64: 99,
 65: 90,
 66: 87,
 67: 80,
 68: 79,
 69: 79,
 70: 109,
 71: 90,
 72: 82,
 73: 72,
 74: 68,
 75: 52,
 76: 76,
 77: 73,
 78: 57,
 79: 44,
 80: 42,
 81: 64,
 82: 58,
 83: 60,
 84: 52,
 85: 59,
 86: 59,
 87: 61,
 88: 59,
 89: 33,
 90: 58,
 91: 40,
 92: 54,
 93: 33,
 94: 45,
 95: 33,
 96: 33,
 97: 42,
 98: 41,
 99: 39,
 100: 36,
 101: 46,
 102: 35,


In [140]:
# good turing (for bigrams)
total_perp = 0
count_sent = 0

for sentence in validation_set:

    count_sent += 1
    if(len(sentence) < 2): continue
    else:
        perp = 1
        n = len(sentence)
        log_perp = 0
        for i in range(1 , n):
            n_gram = (sentence[i-1], sentence[i])

            if(n_gram[0] not in unigrams) or (n_gram not in bigrams):
                log_perp += math.log(len(unigrams))

            else:
                if bigrams[n_gram]>100:
                    log_perp += math.log((unigrams[n_gram[0]])/(bigrams[n_gram]-0.75))
                elif (bigrams[n_gram] ==0):

                    log_perp += math.log((bigram_freq[1]/N0))



                else:
                    log_perp += math.log(((unigrams[n_gram[0]])
                    *bigram_freq[bigrams[n_gram]])/((bigrams[n_gram]+1)
                    *bigram_freq[bigrams[n_gram]+1]))

        log_perp = (1/n)*log_perp
        total_perp += math.exp(log_perp)
print("average perplexity of bigram after turing is: ",total_perp / count_sent)

average perplexity of bigram after turing is:  643.5744591759906


In [141]:

trigram_freq ={}

for frequency in trigrams.values():
    if frequency not in trigram_freq:
        trigram_freq[frequency] = 1
    else:
        trigram_freq[frequency] += 1


trigram_freq = dict(sorted(bigram_freq.items()))

trigram_freq

{1: 548511,
 2: 131693,
 3: 44380,
 4: 25590,
 5: 15691,
 6: 10828,
 7: 7964,
 8: 6031,
 9: 4792,
 10: 3875,
 11: 3155,
 12: 2561,
 13: 2301,
 14: 1995,
 15: 1782,
 16: 1514,
 17: 1403,
 18: 1210,
 19: 1103,
 20: 986,
 21: 900,
 22: 805,
 23: 712,
 24: 689,
 25: 593,
 26: 567,
 27: 514,
 28: 510,
 29: 440,
 30: 433,
 31: 415,
 32: 371,
 33: 349,
 34: 321,
 35: 259,
 36: 312,
 37: 278,
 38: 235,
 39: 244,
 40: 260,
 41: 217,
 42: 214,
 43: 214,
 44: 217,
 45: 183,
 46: 160,
 47: 174,
 48: 180,
 49: 156,
 50: 144,
 51: 160,
 52: 138,
 53: 121,
 54: 137,
 55: 136,
 56: 109,
 57: 110,
 58: 104,
 59: 97,
 60: 112,
 61: 112,
 62: 98,
 63: 99,
 64: 99,
 65: 90,
 66: 87,
 67: 80,
 68: 79,
 69: 79,
 70: 109,
 71: 90,
 72: 82,
 73: 72,
 74: 68,
 75: 52,
 76: 76,
 77: 73,
 78: 57,
 79: 44,
 80: 42,
 81: 64,
 82: 58,
 83: 60,
 84: 52,
 85: 59,
 86: 59,
 87: 61,
 88: 59,
 89: 33,
 90: 58,
 91: 40,
 92: 54,
 93: 33,
 94: 45,
 95: 33,
 96: 33,
 97: 42,
 98: 41,
 99: 39,
 100: 36,
 101: 46,
 102: 35,


In [142]:
# good turing (for trigrams)
total_perp = 0
count_sent = 0

for sentence in validation_set:

    count_sent += 1
    if(len(sentence) < 3): continue
    else:
        perp = 1
        n = len(sentence)
        log_perp = 0
        for i in range(2 , n):
            n_gram = (sentence[i-2], sentence[i-1], sentence[i])

            if(n_gram[0] not in bigrams) or (n_gram not in trigrams):
                log_perp += math.log(len(bigrams))

            else:
                if trigrams[n_gram]>100:
                    log_perp += math.log((bigrams[n_gram[0]])/(trigrams[n_gram]-0.75))
                elif (trigrams[n_gram] ==0):

                    log_perp += math.log((bigram_freq[1]/N0))



                else:
                    log_perp += math.log(((bigrams[n_gram[0]])
                    *bigram_freq[trigrams[n_gram]])/((trigrams[n_gram]+1)
                    *bigram_freq[trigrams[n_gram]+1]))

        log_perp = (1/n)*log_perp
        total_perp += math.exp(log_perp)
print("average perplexity of bigram after turing is: ",total_perp / count_sent)

average perplexity of bigram after turing is:  252658.39010734862


In [143]:

quadgram_freq ={}

for frequency in quadgrams.values():
    if frequency not in quadgram_freq:
        quadgram_freq[frequency] = 1
    else:
        quadgram_freq[frequency] += 1


quadgram_freq = dict(sorted(bigram_freq.items()))

quadgram_freq

{1: 548511,
 2: 131693,
 3: 44380,
 4: 25590,
 5: 15691,
 6: 10828,
 7: 7964,
 8: 6031,
 9: 4792,
 10: 3875,
 11: 3155,
 12: 2561,
 13: 2301,
 14: 1995,
 15: 1782,
 16: 1514,
 17: 1403,
 18: 1210,
 19: 1103,
 20: 986,
 21: 900,
 22: 805,
 23: 712,
 24: 689,
 25: 593,
 26: 567,
 27: 514,
 28: 510,
 29: 440,
 30: 433,
 31: 415,
 32: 371,
 33: 349,
 34: 321,
 35: 259,
 36: 312,
 37: 278,
 38: 235,
 39: 244,
 40: 260,
 41: 217,
 42: 214,
 43: 214,
 44: 217,
 45: 183,
 46: 160,
 47: 174,
 48: 180,
 49: 156,
 50: 144,
 51: 160,
 52: 138,
 53: 121,
 54: 137,
 55: 136,
 56: 109,
 57: 110,
 58: 104,
 59: 97,
 60: 112,
 61: 112,
 62: 98,
 63: 99,
 64: 99,
 65: 90,
 66: 87,
 67: 80,
 68: 79,
 69: 79,
 70: 109,
 71: 90,
 72: 82,
 73: 72,
 74: 68,
 75: 52,
 76: 76,
 77: 73,
 78: 57,
 79: 44,
 80: 42,
 81: 64,
 82: 58,
 83: 60,
 84: 52,
 85: 59,
 86: 59,
 87: 61,
 88: 59,
 89: 33,
 90: 58,
 91: 40,
 92: 54,
 93: 33,
 94: 45,
 95: 33,
 96: 33,
 97: 42,
 98: 41,
 99: 39,
 100: 36,
 101: 46,
 102: 35,


In [144]:
# good turing (for quadgrams)
total_perp = 0
count_sent = 0

for sentence in validation_set:

    count_sent += 1
    if(len(sentence) < 4): continue
    else:
        perp = 1
        n = len(sentence)
        log_perp = 0
        for i in range(3 , n):
            n_gram = (sentence[i-3], sentence[i-2], sentence[i-1], sentence[i])

            if(n_gram[0] not in trigrams) or (n_gram not in quadgrams):
                log_perp += math.log(len(trigrams))

            else:
                if quadgrams[n_gram]>100:
                    log_perp += math.log((trigrams[n_gram[0]])/(quadgrams[n_gram]-0.75))
                elif (quadgrams[n_gram] ==0):

                    log_perp += math.log((bigram_freq[1]/N0))



                else:
                    log_perp += math.log(((trigrams[n_gram[0]])
                    *bigram_freq[quadgrams[n_gram]])/((quadgrams[n_gram]+1)
                    *bigram_freq[quadgrams[n_gram]+1]))

        log_perp = (1/n)*log_perp
        total_perp += math.exp(log_perp)
print("average perplexity of bigram after turing is: ",total_perp / count_sent)

average perplexity of bigram after turing is:  399270.065792176
